In [1]:
import pandas as pd
from vincent import *
from collections import defaultdict
import load_data as load
import vincent

['France', 'Netherlands', 'Portugal', 'Canada', 'Germany', 'Italy', 'United States of America', 'Australia', 'United Kingdom', 'Spain']
['FRA', 'NLD', 'PRT', 'CAN', 'DEU', 'ITA', 'USA', 'AUS', 'GBR', 'ESP']
[4138, 589, 159, 1131, 894, 2261, 50699, 459, 1890, 1811]


In [2]:
def increase_data(key, destination, data):
    if key not in data:
        destination_dict = defaultdict(int)
        destination_dict.setdefault(destination, 1)
        data.setdefault(key, destination_dict)
    else:
        destination_dict = data.get(key)
        if destination not in destination_dict:
            destination_dict.setdefault(destination, 1)
        else:
            destination_dict[destination] += 1

In [24]:
users_data, users_dict = load.get_train_users_data()
month_data = defaultdict(lambda: defaultdict(int))
book_month_data = defaultdict(lambda: defaultdict(int))
gender_data = defaultdict(lambda: defaultdict(int))
age_data = defaultdict(lambda: defaultdict(int))
language_data = defaultdict(lambda: defaultdict(int))
affiliate_provider_data = defaultdict(lambda: defaultdict(int))
first_device_type_data = defaultdict(lambda: defaultdict(int))
first_browser_data = defaultdict(lambda: defaultdict(int))

for user in users_data:
    destination = user[load.USERS_COUNTRY_DESTINATION]

    timestamp_first_active = user[load.USERS_TIMESTAMP_FIRST_ACTIVE]
    year = int(timestamp_first_active[:4])
    month = int(timestamp_first_active[4:6])
    day = int(timestamp_first_active[6:8])

    if destination != 'NDF':
        date_first_booking = user[load.USERS_DATE_FIRST_BOOKING].strip().split('-')
        book_year = int(date_first_booking[0])
        book_month = int(date_first_booking[1])
        book_day = int(date_first_booking[2])
        increase_data(book_month, destination, book_month_data)

    gender = user[load.USERS_GENDER]
    age = user[load.USERS_AGE]
    if age is None or age == '':
        age = '-unknown-'
    else:
        if float(age) >= 100:
            age = '100'
        else:    
            age = str(int(float(age)) / 10 * 10) + '-' + str(int(float(age)) / 10 * 10 + 9)

    language = user[load.USERS_LANGUAGE]
    affiliate_provider = user[load.USERS_AFFILIATE_PROVIDER]
    first_device_type = user[load.USERS_FIRST_DEVICE_TYPE]
    first_browser = user[load.USERS_FIRST_BROWSER]

    increase_data(month, destination, month_data)
    increase_data(gender, destination, gender_data)
    increase_data(age, destination, age_data)
    increase_data(language, destination, language_data)
    increase_data(affiliate_provider, destination, affiliate_provider_data)
    increase_data(first_device_type, destination, first_device_type_data)
    increase_data(first_browser, destination, first_browser_data)


In [25]:
print age_data

defaultdict(<function <lambda> at 0x105a1df50>, {'50-59': defaultdict(<type 'int'>, {'FR': 297, 'NL': 29, 'PT': 15, 'CA': 71, 'DE': 49, 'IT': 151, 'US': 2787, 'NDF': 4402, 'other': 435, 'AU': 36, 'GB': 149, 'ES': 85}), '20-29': defaultdict(<type 'int'>, {'FR': 729, 'NL': 128, 'PT': 31, 'CA': 197, 'DE': 210, 'IT': 439, 'US': 11096, 'NDF': 12919, 'other': 1548, 'AU': 93, 'GB': 341, 'ES': 429}), '60-69': defaultdict(<type 'int'>, {'FR': 144, 'NL': 21, 'PT': 5, 'CA': 34, 'DE': 36, 'IT': 79, 'US': 1453, 'NDF': 2038, 'other': 250, 'AU': 14, 'GB': 82, 'ES': 49}), '-unknown-': defaultdict(<type 'int'>, {'FR': 1131, 'NL': 140, 'PT': 37, 'CA': 292, 'DE': 182, 'IT': 636, 'US': 11996, 'NDF': 52688, 'other': 2076, 'AU': 87, 'GB': 447, 'ES': 460}), '10-19': defaultdict(<type 'int'>, {'FR': 51, 'NL': 8, 'PT': 1, 'CA': 19, 'DE': 16, 'IT': 25, 'US': 424, 'NDF': 616, 'other': 90, 'AU': 5, 'GB': 21, 'ES': 18}), '0-9': defaultdict(<type 'int'>, {'FR': 2, 'DE': 1, 'IT': 2, 'US': 6, 'NDF': 43, 'other': 1, '

In [32]:
def visualization(data, is_large, x_desc, y_desc, title):
    index = []
    value = []
    key_list = data.keys()
    key_list.sort()
    for key in key_list:
        index.append(key)
        datum = data.get(key)
        if is_large:
            value.append({'NDF': datum['NDF'], 'US': datum['US'], 'other': datum['other']})
        else:
            value.append({'AU': datum['AU'], 'CA': datum['CA'], 'DE': datum['DE'], 
                          'ES': datum['ES'], 'FR': datum['FR'], 'GB': datum['GB'],
                          'IT': datum['IT'], 'NL': datum['NL'], 'PT': datum['PT']})

    df = pd.DataFrame(value, index=index)
    vis = GroupedBar(df)
    vis.axis_titles(x=x_desc, y=y_desc)
    vis.width = 1000
    vis.legend(title=title)
    vis.colors(brew='Pastel1')
    vis.display()
    

In [54]:
vincent.core.initialize_notebook()
visualization(first_browser_data, False, 'First browser', '#users', 'First browser and #users distribution')